In [1]:
#Install Pyspark
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version
# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark
# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"
# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    3653882|R3O9SGZBVQBV76|B00FALQ1ZC|     937001370|Invicta Women's 1...|         Watches|          5|            0|          0|   N|                Y|          Five Stars|Absolutely love t...| 2015-08-31|
|         US|   14661224| RKH8BNC3L5DLF|B00D3RGO20|     484010722|Kenneth Cole New ...|         Watches|          5|    

In [74]:
from pyspark.sql.functions import col, avg

In [10]:
votes_df = df.filter(df["total_votes"] >= "20")
votes_df.show(20,truncate=False)

+-----------+-----------+--------------+----------+--------------+---------------------------------------------------------------------------------------------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [15]:
#votes_df_2 = df.filter(df["helpful_votes"]/["total_votes"] >= "1")
#votes_df_2 = df.filter(df["helpful_votes"/"total_votes"] >= "1")
votes_df_2 = df.filter(df["helpful_votes"] / df["total_votes"] >= "0.5")
votes_df_2.show(5,truncate=False)

+-----------+-----------+--------------+----------+--------------+--------------------------------------------------------------------------------------+----------------+-----------+-------------+-----------+----+-----------------+------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+
|marketplace|customer_id|review_id     |product_id|product_parent|product_title                                                                         |product_category|star_rating|help

In [16]:
votes_df_2.filter(votes_df_2["vine"] == "Y").show(20)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   52229317| R6EMI5RKW73N3|B00QK2HAFK|      78116254|Armitron Men's 20...|         Watches|          4|            1|          2|   Y|                N|Great Watch for t...|This is a good lo...| 2015-08-31|
|         US|   52408013| R2IZD1134PGIP|B003XOT3T4|     368215170|Swiss Legend Men'...|         Watches|          4|    

In [17]:
votes_df_2.filter(votes_df_2["vine"] == "N").show(20)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   27324930|R2HLE8WKZSU3NL|B00DKYC7TK|     361166390|Ritche 22mm Black...|         Watches|          2|            1|          1|   N|                Y|           Two Stars|           Scratches| 2015-08-31|
|         US|   11811565| RB2Q7DLDN6TH6|B00WM0QA3M|     549298279|Fossil Women's ES...|         Watches|          5|    

In [80]:
from pyspark.sql.functions import col, avg

#Total number of reviews

#df.filter(df["review_id"] != 'NaN').count()

df[df["review_id"] != 'NaN'].count()


960872

In [68]:
#Total number of 5 star reviews
#total5StarReviews = votes_df_2.filter(votes_df_2["star_rating"] == '5').count()

total5StarReviews = df.filter(df["star_rating"] == '5').count()
total5StarReviews


571627

In [70]:
#Total number of 5 star reviews (paid)
totalPaid5StarReviews = df.filter((df["star_rating"] == '5') & (df["vine"] == 'Y')).count() 

totalPaid5StarReviews

605

In [72]:
percentagePaid5StarReviews = (totalPaid5StarReviews / total5StarReviews) * 100

percentagePaid5StarReviews

0.10583824766849712

In [73]:
#Total number of 5 star reviews (Unpaid)
totalUnPaid5StarReviews = df.filter((df["star_rating"] == '5') & (df["vine"] == 'N')).count()

totalUnPaid5StarReviews

571022

In [77]:
percentageUnPaid5StarReviews = (totalUnPaid5StarReviews / total5StarReviews) * 100

percentageUnPaid5StarReviews


99.8941617523315